In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from google.colab import drive

In [8]:
# Load your dataset
drive.mount('/content/gdrive')
data = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/dataset_updated2.xlsx')

# Extract relevant columns (S8_cur, S4_cur, S2_cur, S1_cur, V_cur, Sp_cur, I, cycle)
selected_columns = ['S8_cur', 'S4_cur', 'S2_cur', 'S1_cur', 'V_cur', 'Sp_cur', 'I','cycle']
filter_columns = ['S8_cur', 'S4_cur', 'S2_cur', 'S1_cur', 'V_cur', 'Sp_cur']
I_value = 1.7
cycle_no = 2   # Might need to link it with dharshannan's microcycling folder
data = data[selected_columns]
filtered_data = data[data['I'] == I_value]

In [17]:
# Identify unique cycles
unique_cycles = filtered_data['cycle'].unique()

# Create an empty dictionary to store scaled and sequenced data for each cycle
data_by_cycle = {}
sequence_length = 200
horizon = 50
X = []
y = []

# Loop through each unique cycle and process it separately
for cycle in unique_cycles:
  cycle_data = filtered_data[filtered_data['cycle'] == cycle]

  # Normalize the data for this cycle
  scaler = MinMaxScaler()
  scaled_data = scaler.fit_transform(cycle_data[filter_columns])

  # Create sequences and corresponding target values
  X_cycle, y_cycle = [], []

  for i in range(len(cycle_data) - sequence_length - horizon + 1):
      X_cycle.append(cycle_data.iloc[i:i+sequence_length].values)
      y_cycle.append(cycle_data.iloc[i+sequence_length:i+sequence_length+horizon]['V_cur'].values)

  X.append(X_cycle)
  y.append(y_cycle)


X_train_combined = []
y_train_combined = []

for cycle in unique_cycles:
  X_train, X_test, y_train, y_test = train_test_split(X[cycle], y[cycle], test_size=0.2, random_state=42)
  X_train_combined = np.concatenate(X_train, axis=0)
  y_train_combined = np.concatenate(y_train, axis=0)


In [19]:
# Define LSTM model architecture
def create_lstm_model(input_shape, output_shape):
    model = Sequential()
    model.add(LSTM(units=32, input_shape=input_shape))
    model.add(Dense(units=output_shape))
    model.compile(optimizer='adam', loss='mse')  # You can choose an appropriate loss function
    return model

# Specify the input shape and output shape based on your data
input_shape = (sequence_length, len(filter_columns))
output_shape = horizon  # Number of timesteps to forecast into the future

# Create and compile the LSTM model
model = create_lstm_model(input_shape, output_shape)

# Train the model with the combined training data from all cycles
model.fit(X_train_combined, y_train_combined, epochs=50)  # You can adjust the number of epochs

ValueError: ignored